In [ ]:
# default_exp config

In [ ]:
#hide
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel/"

Mounted at /content/drive
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
#hide
#!pip install nbdev
!nbdev_build_lib

Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.


In [ ]:
#hide
%run "_Synch.ipynb"

# Configuration
> DRY conviences

In [ ]:
# export

import json, os, shutil

class Defaults:
  """
  Makes certain variables are very accessible across the repository. The names 
  of the variables and their respective values are stored in JSON format in 
  `./ModelAssistedLabel config.json`  

  Functions defined here are also available across this project.
  """

  def __init__(self, config_file="ModelAssistedLabel config.json"):
    self.config_file=config_file

    with open(config_file, "r") as f:
      indata = (json.load(f))
    for k,v in indata.items():
      self.__dict__[k] = v

  def prepare_YOLOv5():
    """
    * Clone repository if the YOLOv5 directory does not exist.
    * Install requirements.txt
    * Check that GPU is enabled.
    """
    # safety for re-executions
    if not os.path.exists("yolov5"):
      # clone YOLOv5 and reset to a specific git checkpoint that has been verified working
      os.system("git clone https://github.com/ultralytics/yolov5")  # clone repo
      os.system("git reset --hard 68211f72c99915a15855f7b99bf5d93f5631330f") # standardize models

    # enter the yolov5 directory
    os.chdir("yolov5")

    # install dependencies as necessary
    os.system("pip install -qr requirements.txt")  # install dependencies (ignore errors)
    import torch

    from IPython.display import Image, clear_output  # to display images
    # from utils.google_utils import gdrive_download  # to download models/datasets

    clear_output()

    if torch.cuda.is_available():
      print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0)))
    else:
      raise Exception("You need to enable your GPU access to this runtime environment")

    # return to parent directory
    os.chdir("..")

  def _itername(pre, post=""):
    """If function terminates, returns the lowest conflict-free file path 
    formatted as '{pre}X{post}' where X is the string representation of a natural
    number
    
    args:
      pre: filename before the counter
      post: filename after the counter

    returns:
      A unique structured filename
    """
    counter = 0
    while True:
      counter += 1
      fpath = f'{pre}{counter}{post}'
      if not os.path.exists(fpath):
        return fpath
  
  def __hard_reset_test_dir__(datadump, keep_folder=False):
    """
    Helpful to be able to delete folders because I want to avoid name conflicts.

    Args:
      datadump: test directory. all contenst are subject to deletion
      keep_folder: if False, will also delete the folder itself.
    """
    if os.path.exists(datadump):
      shutil.rmtree(datadump)
      print(f"deleted `{datadump}`")
    else:
      print(f"`{datadump}`` did not exist")

    if keep_folder:
      os.makedirs(datadump)
      print(f"making `{datadump}`")
      assert os.path.exists(datadump)
      assert len(os.listdir(datadump)) == 0
    else:
      print(f"not making `{datadump}`")
      assert not os.path.exists(datadump)

  def read_json(self, json_file= None):
    """Reads and returns the value of a json file
    
    Args:
      json_file: the path to a json file. By default None is `self.config_file`

    Returns:
      the contents of the json file
    """
    if json_file is None:
      json_file = self.config_file
    with open(json_file) as config:
      raw = config.readlines()[0]
      return json.loads(raw)

  def save(self):
    "save changes made to attributes"
    with open(self.config_file, "w") as config_file:
      json.dump(self.__dict__, config_file)


make sure that autonaming works

In [ ]:
datadump = "ipynb_tests/00_config_datadump"
extension = ".text"
Defaults.__hard_reset_test_dir__(datadump, keep_folder=True)

for i in range(3):
  next_filename = Defaults._itername(pre = f"{datadump}/Defaults (", 
                                     post = f"){extension}")
  with open(next_filename, "w") as outfile:
    outfile.writelines("<data>")

for i in range(3):
  next_filename = Defaults._itername(pre = f"{datadump}/Version - ", 
                                     post = "")
  with open(next_filename, "w") as outfile:
    outfile.writelines("<data>")

files = os.listdir(datadump)
assert len(files) == 6
files

deleted `ipynb_tests/00_config_datadump`
making `ipynb_tests/00_config_datadump`


['Defaults (1).text',
 'Defaults (2).text',
 'Defaults (3).text',
 'Version - 1',
 'Version - 2',
 'Version - 3']

**Default Values** are stored in "ModelAssistedLabel config.json"

Every time the class is called, the config file is re-read for changes.

Currently, the following attributes are then dynamically assigned to the newly-created `Default` object.

* root *(parent folder of YOLOv5 repo)*
* resource_map *(defines images as ".jpg" and labels as ".txt")*
* split_ratio *(by default, 70/20/10 split of train/valid/test.)*
* data_yaml *(from YOLOv5 repo)*
* trainer template *(from YOLOv5 repo)*

This data is generated dynamically. 



In [ ]:
import json
d = Defaults()
data = d.read_json()
for k,v in data.items():
  print("Attribute:", k, "\n\t- Type:", type(v))

reading defaults from: ModelAssistedLabel config.json
Attribute: config_file 
	- Type: <class 'str'>
Attribute: root 
	- Type: <class 'str'>
Attribute: split_ratio 
	- Type: <class 'dict'>
Attribute: data_yaml 
	- Type: <class 'str'>
Attribute: resource_map 
	- Type: <class 'dict'>
Attribute: trainer_template 
	- Type: <class 'str'>
